## 概述
程序计数器、虚拟机栈、本地方法栈，生存周期与对应的线程一致。
方法区内存回收：基本上是编译器确定的，方法结束或者线程结束时，内存自然就跟随着回收
## 判断对象是否需要回收的算法
### 引用计数法 ——给对象添加一个计数器，若增加一个对象的引用，则计数器加一，饮用失效时，计数器减一。当计数器值为0时，说明对象不可再被使用。
* 优点：判定简单，效率高
* 缺点：难以解决循环引用的问题。

### 可达性分析算法
+ 基本思想： 通过一系列的称为“GC Roots”的对象作为起始点，从这些节点开始向下搜索，搜索所走过的路径称为引用链（Reference Chain），当一个对象到GC Roots没有任何引用链相连
（用图论的话来说，就是从GC Roots到这个对象不可达）时，则证明此对象是不可用的。
+ 可作为GC ROOT的对象：
  * 虚拟机栈（栈帧中的本地变量表）中引用的对象。
  * 方法区中类静态属性引用的对象。
  * 方法区中常量引用的对象。
  * 本地方法栈中JNI（即一般说的Native方法）引用的对象。
  
### 引用
* 强引用
* 软引用
* 弱引用
* 虚引用

### 对象是否会被回收

### 回收方法区
 方法区（hotSpot中的永久带）垃圾回收效率较低
 永久带垃圾回收主要内容
* 废弃常量——与回收java堆中对象相似。
* 无用的类——要满足一下几个条件，才算是无用的类
 1.  该类所有的实例都已经被回收，也就是Java堆中不存在该类的任何实例。
 1.  加载该类的ClassLoader已经被回收。
 1.  该类对应的java.lang.Class对象没有在任何地方被引用，无法在任何地方通过反射访问该类的方法。 


 在大量使用反射、动态代理、CGLib等ByteCode框架、动态生成JSP以及OSGi这类频繁自定义ClassLoader的场景都需要虚拟机具备类卸载的功能，以保证永久代不会溢出。

# 垃圾收集算法
## 标记-清除算法
* 算法分为两个阶段
 1. 标记——首先标记出所有需要回收的对象，
 2. 清除——在标记完成后统一回收所有被标记的对象
* 不足：
    2. 效率：标记与清楚地效率都比较低
    2. 内存碎片。收集后会产生大量内存碎片。导致程序运行时，无法找到大块内存，引发再一次垃圾收集。
## 复制算法
* 方法—— 将内存分为大小相同的两块，每次只使用一块。内存用尽时，就把仍存活的对象移到内存的另一块，然后将已使用的内存一次清除。
* 缺点：
     1.内存缩小为原先的一半，代价太高。
     1. 对象存活率较高时就要进行较多的复制操作，效率将会变低。
     1. 如果不想浪费50%的空间，就需要有额外的空间进行分配担保，以应对被使用的内存中所有对象都100%存活的极端情况，所以在老年代一般不能直接选用这种算法。
* 实际应用：现在商用虚拟机采用的垃圾回收算法。
    1. 因为新生对象98%都是会很快死掉，所以不需要1:1的比例来划分内存。而是将内存分为一块较大的Eden区，和两块较小的Survivor区。
    2. 每次使用Eden和一块Survivor。回收时，将存活的对象移到另外一块Survivor上，然后回收使用的Eden和Survivor空间。HotSpot虚拟机默认Eden和Survivor的大小比例是8:1。
    3. 当Survivor空间不够用时，需要依赖其他内存（这里指老年代）进行分配担保（Handle Promotion）

## 标记整理算法

根据老年代的特点，有人提出了另外一种“标记-整理”（Mark-Compact）算法，标记过程
仍然与“标记-清除”算法一样，但后续步骤不是直接对可回收对象进行清理，而是让所有存
活的对象都向一端移动，然后直接清理掉端边界以外的内存。

## 分代收集算法（当前虚拟机采用的垃圾收集算法）
* 思想：根据对象存活周期的不同将内存划分为几块。一般是把Java堆分为新生代和老年代，这样就可以根据各个年代的特点采用最适当的收集算法。
* 在新生代中，每次垃圾收集时都发现有大批对象死去，只有少量存活，那就选用复制算法，只需要付出少量存活对象的复制成本就可以完成收集。
* 而老年代中，因为对象存活率高、没有额外空间对它进行分配担保，就必须使用“标记—清理”或者“标记—整理”算法来进行回收。

# HotSpot算法实现
## 枚举根节点
 可达性分析的局限性：
 * 时间效率低： 可作为GC Roots的节点主要在全局性的引用（例如常量或类静态属性）与执行上下文（例如栈帧中的本地变量表）中，现在很多应用仅仅方法区就有数百兆，如果要逐个检查这里面的引用，那么必然会消耗很多时间。
 * GC停顿 这项分析工作必须在一个能确保一致性的快照中进行。这导致GC进行时必须停顿所有的java线程。
 
 由于目前的主流Java虚拟机使用的都是准确式GC，所以当执行系统停顿下来后，并不需要一个不漏地检查完所有执行上下文和全局的引用位置，虚拟机应当是有办法直接得知哪些地方存放着对象引用。在HotSpot的实现中，是使用一组称为OopMap的数据结构来达到这个目的的，在类加载完成的时候，HotSpot就把对象内什么偏移量上是什么类型的数据计算出来，在JIT编译过程中，也会在特定的位置记录下栈和寄存器中哪些位置是引用。这样，GC在扫描时就可以直接得知这些信息了。下面的代码清单3-3是HotSpot Client VM生成的一段String.hashCode（）方法的本地代码，可以看到在0x026eb7a9处的call指令有OopMap记录，它指明了EBX寄存器和栈中偏移量为16的内存区域中各有一个普通对象指针（Ordinary Object Pointer）的引用，有效范围为从call指令开始直到0x026eb730（指令流的起始位置）+142（OopMap记录的偏移量）=0x026eb7be，即hlt指令为止。
## 安全点
## 安全区域

# 5.垃圾收集器
JDK1.7 update14后所有的垃圾收集器：

## 新生代
### Serial收集器
* 最古老的垃圾收集器
* 新生代采取复制算法，老年代采取标记-整理算法。
* 这个收集器是一个单线程的收集器，但它的“单线程”的意义并不仅仅说明它只会使用一个CPU或一条收集线程去完成垃圾收集工作，更重要的是在它进行垃圾收集时，必须暂停其他所有的工作线程，直到它收集结束。
* 虚拟机运行在Client模式下的默认新生代收集器
* 优点：简单而高效，限定单个CPU的环境，没有线程切换的开销。
* 缺点：，在用户不可见的情况下把用户正常工作的线程全部停掉，并且单线程收集垃圾，可能造成响应性降低。
## ParNew收集器
* 是Serial收集器的多线程版本
* 新生代采取复制算法，老年代采取标记-整理算法。
* 使用多条线程进行垃圾收集之外，其余行为包括Serial收集器可用的所有控制参数（例如：-XX：SurvivorRatio、-XX：PretenureSizeThreshold、-XX：HandlePromotionFailure等）、收集算法、Stop The World、对象分配规则、回收策略等都与Serial收集器完全一样。
* 运行在Server模式下的虚拟机中首选的新生代收集器，其中有一个与性能无关但很重要的原因是，除了Serial收集器外，目前只有它能与CMS收集器配合工作。
* 单CPU的效果不比Serial收集器好，效果随着CPU个数增加而提升。

* ps：并行收集器与并发收集器
 1. 并行（Parallel）：指多条垃圾收集线程并行工作，但此时用户线程仍然处于等待状态。
 2. 并发（Concurrent）：指用户线程与垃圾收集线程同时执行（但不一定是并行的，可能会交替执行），用户程序在继续运行，而垃圾收集程序运行于另一个CPU上。
### Parallel Scavenge收集器
* 使用复制算法的并行新生代收集器
* 目标：达到可控制的预期吞吐量（CPU运行用户代码的时间/CPU运行总时间）
* 提供两个参数精确控制吞吐量
  1. 控制最大垃圾收集停顿时间的-XX：MaxGCPauseMillis参数——大于0的毫秒数，收集器将尽可能地保证内存回收花费的时间不超过设定值
  2. 直接设置吞吐量大小的-XX：GCTimeRatio参——是一个大于0且小于100的整数，也就是垃圾收集时间占总时间的比率，相当于是吞吐量的倒数
  3. 自适应调节策略开关参数：-XX：+UseAdaptiveSizePolicy——当这个参数打开之后，就不需要手工指定新生代的大小（-Xmn）、Eden与Survivor区的比例（-XX：SurvivorRatio）、晋升老年代对象年龄（-XX：PretenureSizeThreshold）等细节参数了，虚拟机会根据当前系统的运行情况收集性能监控信息，动态调整这些参数以提供最合适的停顿时间或者最大的吞吐量，这种调节方式称为GC自适应的调节策略（GC Ergonomics）。
## 老年代
### CMS（Concurrent Mark Sweep）
* 获取最短回收停顿时间为目标，集中在互联网站或者B/S系统的服务端（注重响应效率）
* 采用标记清除算法，过程分为四个步骤
    1.初始标记（需要停止所有用户线程）：标记一下GC Roots能直接关联到的对象，速度很快
    1.并发标记
    3.重新标记（需要停止所有用户线程）：修正并发标记期间因用户程序继续运作而导致标记产生变动的那一部分对象的标记记录，这个阶段的停顿时间一般会比初始标记阶段稍长一些，但远比并发标记的时间短
    4.并发清除
    
* 优点： 并发收集、低停顿
* 缺点：
    1. 采用标记清除算法，会产生内存碎片。分配大对象时，无法找到大的连续内存空间，会导致触发一次Full GC。
    
    2. CMS收集器对CPU资源非常敏感：会因为占用了一部分线程（或者说CPU资源）而导致应用程序变慢，总吞吐量会降低。。CMS默认启动的回收线程数是（CPU数量+3）/4，也就是当CPU在4个以上时，并发回收时垃圾收集线程不少于25%的CPU资源，并且随着CPU数量的增加而下降。但是当CPU不足4个（譬如2个）时，CMS对用户程序的影响就可能变得很大，如果本来CPU负载就比较大，还分出一半的运算能力去执行收集器线程，就可能导致用户程序的执行速度忽然降低了50%，其实也让人无法接受。
    为了应付这种情况，虚拟机提供了一种称为“增量式并发收集器”（Incremental Concurrent Mark Sweep/i-CMS）的CMS收集器变种，但实践中增量时的CMS收集器效果很一般，在目前版本中，i-CMS已经被声明为“deprecated”， 不再提倡用户使用。
    3. CMS收集器无法处理浮动垃圾：可能出现“Concurrent ModeFailure”失败而导致另一次Full GC的产生。
      * 浮动垃圾：并发清理阶段产生的，本次收集处理阶段中没办法处理调的垃圾。
      * 由于在垃圾清理阶段用户线程还需要运行，所以需要预留足够内存给用户线程，因此CMS不能像其他老年代垃圾收集器，等待老年代几乎完全被填满再进行收集，需要预留一部分空间提供并发收集时的程序运作使用。这部分空间大小可以通过设置参数-XX：CMSInitiatingOccupancyFraction（1.5为68%，1.6之后为92%，太大容易导致大量Concurrent ModeFailure）。若CMS运行期间无法提供预留内存，则引发Concurrent ModeFailure，虚拟机启动后备预案：临时启动Serial Old来进行垃圾收集。
### Serial Old（MSC）
* Serial 的老年代版本
* 用途：
    1. JDK1.5之后与Parallel Scavenge搭配使用
    2. 作为CMS的后备预案。
### Parallel Old
* Parallel Scavenge的老年代版本，使用多线程和标记-整理算法
* 注重吞吐量以及CPU资源敏感的场合，都可以优先考虑Parallel Scavenge加Parallel Old收集器
##新老通用
###G1
* 面向服务端应用的垃圾收集器，特点如下：
 1. 并行与并发——G1能充分利用多CPU、多核环境下的硬件优势，使用多个CPU（CPU或者CPU核心）来缩短Stop-The-World停顿的时间，部分其他收集器原本需要停顿Java线程执行的GC动作，G1收集器仍然可以通过并发的方式让Java程序继续执行。
 2. 分代收集——与其他收集器一样，分代概念在G1中依然得以保留。虽然G1可以不需要其
他收集器配合就能独立管理整个GC堆，但它能够采用不同的方式去处理新创建的对象和已
经存活了一段时间、熬过多次GC的旧对象以获取更好的收集效果。
 3. 空间整合
 4. 可预测的停顿
 
* 运行步骤
 1. 初始标记
 2. 并发标记
 3. 最终标记
 4. 筛选回收
 
##GC日志


#内存分配与回收策略
##对象优先在Eden区
* 大多数情况，对象优先在新生代Eden区分配，当Eden区没有足够空间，虚拟机会发起一次minor GC
    PS：
    1. minor GC： 新生代的垃圾收集动作，因为新生代对象一般朝生夕灭，所以Monor GC很频繁，回收速度也较快
    2. Major GC/Full GC 发生在老年代的GC。出现Major GC一般会伴随着至少一次minor GC（不一定，parallel scavenge有直接Full GC的策略选择过程）。Major GC的速度一般比Minor GC慢十倍以上。

##大对象直接进入老年代
##长期存活的对象进入老年代





































